In [72]:
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import math

In [17]:
def read_from_file(file_name):
    data = []
    f = open(file_name, mode="r", encoding="utf-8")
    for line in f.readlines():
        data.append({"score":int(line[0:1]), "comment":line[2:]})
    f.close()
    return pd.DataFrame(data)

train = read_from_file("./collectdata/train.txt")
test = read_from_file("./collectdata/valid.txt")

In [47]:
train.groupby("score").count()

,comment
score,
1,45
2,47
3,101
4,349
5,1257


In [88]:
def train_model(**kwargs):
    
    def calc_probabilities_in_class(data):
        bag_of_words = {}
        for index, row in data.iterrows():
            words = nltk.word_tokenize(row['comment'])
            for word in words:
                word_l = word.lower()
                if not word_l in bag_of_words:
                    bag_of_words[word_l] = 1
                else:
                    bag_of_words[word_l] = bag_of_words[word_l] + 1
        return bag_of_words
    
    probabilities = {}
    for key in kwargs:
        probabilities[key] = calc_probabilities_in_class(kwargs[key])
    return probabilities


def predict_comment(model, comment):
    
    def comment_log_probabitlity(words, class_model, total_words):
        denominator = sum(class_model.values()) + len(class_model)
        log_prob = sum(class_model.values()) / total_words
        for word in words:
            n = class_model[word] if word in class_model else 0 
            log_prob += math.log((n + 1)/denominator)
        return log_prob
        
    log_probablitites = {}
    words = nltk.word_tokenize(comment)
    
    total_words = 0
    for key in model:
        total_words += sum(model[key].values())
        
    for key in model:
        log_probablitites[key] = comment_log_probabitlity(words, model[key], total_words)
    return log_probablitites


def predict_on_set(model, test_set):

    for index, row in test_set.iterrows():
        max_prob = float('-inf')
        max_key = ""
        probs = predict_comment(model, row["comment"])
        
        for key_class in probs:
            if probs[key_class] > max_prob:
                max_prob = probs[key_class]
                max_key = key_class
                
        print(max_key, ":", row["score"])
        
    

In [89]:
model_2 = train_model(positive = train[train["score"] > 3], negative = train[train["score"] < 3])
predict_on_set(model, test)

negative : 5
negative : 5
negative : 2
negative : 1
negative : 5
negative : 3
negative : 5
negative : 5
negative : 4
negative : 5
negative : 5
negative : 4
negative : 5
negative : 5
negative : 5
negative : 5
negative : 5
negative : 4
positive : 4
negative : 5
negative : 5
negative : 5
positive : 5
positive : 5
negative : 5
negative : 4
negative : 5
positive : 5
negative : 5
negative : 5
negative : 3
negative : 5
negative : 5
negative : 5
negative : 5
negative : 5
negative : 2
negative : 2
negative : 5
negative : 4
negative : 5
negative : 5
positive : 4
negative : 5
negative : 5
negative : 5
negative : 4
negative : 2
negative : 5
negative : 4
negative : 4
negative : 5
negative : 5
negative : 4
negative : 5
negative : 5
negative : 5
negative : 4
negative : 5
negative : 4
negative : 5
negative : 4
negative : 5
negative : 5
negative : 3
negative : 5
positive : 5
negative : 5
negative : 5
negative : 5
positive : 5
negative : 5
negative : 5
negative : 4
negative : 5
negative : 3
negative : 5